## Input을 음성파일로 넣는 Ver ##

In [2]:
from faster_whisper import WhisperModel

model_size = "large-v3"

# Run on GPU with FP16
model = WhisperModel(model_size, device="cuda", compute_type="float16")

# or run on GPU with INT8
# model = WhisperModel(model_size, device="cuda", compute_type="int8_float16")
# or run on CPU with INT8
# model = WhisperModel(model_size, device="cpu", compute_type="int8")

# segments, info = model.transcribe("C:/Users/cy/git/kospeech/data/wavs_full/154.wav", beam_size=5)

# print("Detected language '%s' with probability %f" % (info.language, info.language_probability))

# for segment in segments:
#     print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))
#     print(type(segment.text))

## Input을 마이크로 입력하는 코드 Ver ##

In [4]:
import pyaudio
import wave
from faster_whisper import WhisperModel

def record_audio(file_path, record_seconds=5, sample_rate=16000, chunk=1024, channels=1):
    # 오디오 설정
    audio_format = pyaudio.paInt16  # 16-bit 해상도
    p = pyaudio.PyAudio()  # PyAudio 객체 생성

    # 오디오 스트림 열기
    stream = p.open(format=audio_format,
                    channels=channels,
                    rate=sample_rate,
                    input=True,
                    frames_per_buffer=chunk)

    print("Recording...")  # 녹음 시작 알림
    frames = []

    # 지정된 시간 동안 오디오 데이터 읽기
    for _ in range(0, int(sample_rate / chunk * record_seconds)):
        data = stream.read(chunk)
        frames.append(data)

    print("Finished recording.")  # 녹음 종료 알림

    # 스트림 정지 및 닫기
    stream.stop_stream()
    stream.close()
    p.terminate()

    # WAV 파일로 저장
    wf = wave.open(file_path, 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(audio_format))
    wf.setframerate(sample_rate)
    wf.writeframes(b''.join(frames))
    wf.close()


In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [5]:
from faster_whisper import WhisperModel

model_size = "large-v3"


def transcribe_audio(file_path, model_size="large-v3", device="cuda", compute_type="float16"):
    # Whisper 모델 로드
    model = WhisperModel(model_size, device=device, compute_type=compute_type)
    
    # 음성 파일 변환 (transcribe)
    segments, info = model.transcribe(file_path, beam_size=5)

    # 감지된 언어와 그 확률 출력
    # print("Detected language '%s' with probability %f" % (info.language, info.language_probability))

    # 각 음성 구간에 대해 텍스트 출력
    for segment in segments:
        print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))
        print(type(segment.text))

if __name__ == "__main__":
    audio_file_path = "recorded_audio.wav"
    record_audio(audio_file_path, record_seconds=5)  # 마이크로부터 5초간 녹음
    transcribe_audio(audio_file_path)  # 녹음된 파일을 Whisper 모델로 변환


Recording...
Finished recording.
[0.00s -> 5.00s]  안녕하세요.
<class 'str'>
